In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# custom packages
from ratsimulator import Agent, trajectory_generator, batch_trajectory_generator
from ratsimulator.Environment import Rectangle

import sys
sys.path.append("../src") if "../src" not in sys.path else None # avoid adding multiple relave paths to sys.path

from Brain import Brain
from Models import SorscherRNN
from methods import *

### Set parameters and initialise

In [ ]:
"""
# Sorscher params
options.save_dir = '/mnt/fs2/bsorsch/grid_cells/models/'
options.n_steps = 100000      # number of training steps
options.batch_size = 200      # number of trajectories per batch
options.sequence_length = 20  # number of steps in trajectory
options.learning_rate = 1e-4  # gradient descent learning rate
options.Np = 512              # number of place cells
options.Ng = 4096             # number of grid cells
options.place_cell_rf = 0.12  # width of place cell center tuning curve (m)
options.surround_scale = 2    # if DoG, ratio of sigma2^2 to sigma1^2
options.RNN_type = 'RNN'      # RNN or LSTM
options.activation = 'relu'   # recurrent nonlinearity
options.weight_decay = 1e-4   # strength of weight decay on recurrent weights
options.DoG = True            # use difference of gaussians tuning curves
options.periodic = False      # trajectories with periodic boundary conditions
options.box_width = 2.2       # width of training environment
options.box_height = 2.2      # height of training environment
"""

# Environment params
boxsize = (2.2, 2.2)
origo = (0,0)
soft_boundary = 0.2
# Brain params
npcs = 512 # as used in Sorscher model
sigma = 0.12
# Training data (Agent) params
batch_size = 64
seq_len = 20
angle0 = None # random
p0 = None     # random
# Agent/random walk parameters
dt = 0.02
sigma = 5.76 * 2
b = 0.13 * 2 * np.pi
mu = 0
# Model params
Ng=4096
Np=npcs # defined for Brain already
weight_decay=1e-4
nonlinearity="relu"
lr=1e-4 # 1e-3 is default for Adam()

# can also select cuda:0 or cuda:1 f.eks when multiple gpus
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
num_workers = 2 # number of workers to load data (in parallel)

In [ ]:
# Init Environment
env = Rectangle(boxsize=boxsize, soft_boundary=soft_boundary)
# Init brain
brain = Brain(env, npcs, sigma)
# Init training data
dataset = Dataset(brain=brain, num_workers=num_workers, batch_size=batch_size, environment=env, seq_len=seq_len, angle0=angle0, p0=p0, dt=dt, sigma=sigma, b=b, mu=mu)

In [ ]:
# Init Environment
env = Rectangle(boxsize=boxsize, soft_boundary=soft_boundary)
# Init brain
brain = Brain(env, npcs, sigma)
# Init training data
btg = batch_trajectory_generator(batch_size, env, seq_len, angle0, p0, dt=dt, sigma=sigma, b=b, mu=mu)
# model init
model = SorscherRNN(2,Ng,Np,weight_decay,nonlinearity)
model.to(device)

In [1]:
torch.nn.ReLU(torch.zeros(2))

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
def data_generator(btg, brain):
    while True:
        pos, vel = next(btg)
        
        #labels = brain(pos) # from euclidean positions to place-cell positions
        labels = brain.softmax_response(pos[:,-1])
        init_pos = brain.softmax_response(pos[:,0])
        #init_pos, labels = labels[:,0], labels[:,1:] # "next" pos is label
        
        # cast to torch tensors
        init_pos = torch.tensor(init_pos, dtype=torch.float32, device=device)
        vel = torch.tensor(vel[:,1:], dtype=torch.float32, device=device) # discard first velocity -> always 0 (initial vel)
        labels = torch.tensor(labels, dtype=torch.float32, device=device)
        yield (vel, init_pos), labels
        
# Initialise data generator
dg = data_generator(btg,brain)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, amsgrad=False)
model.train(trainloader=dg, optimizer = optimizer, nepochs=10, nsteps=2)

In [ ]:
from time import time
import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, amsgrad=False)
nepochs, nsteps = 10, 2
trainloader=dg

trainloading_time = np.zeros(nepochs)
propagation_time = np.zeros(nepochs)
loss_history = []
pbar = tqdm.tqdm(range(nepochs))
for epoch in pbar:
    # generic torch training loop
    running_loss = 0.0
    for _ in range(nsteps):
        # get the inputs; data is a list of [inputs, labels]
        start = time()
        inputs, labels = next(trainloader)
        trainloading_time[epoch] += (time() - start)

        start = time()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        predictions = model(inputs, softmax=True)
        loss = model.loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        propagation_time[epoch] += (time() - start)

        running_loss = loss.item()

    loss_history.append(running_loss / nsteps)
    pbar.set_description(f"Epoch={epoch}, loss={loss_history[-1]}")


In [ ]:
trainloading_time, propagation_time

# Specify and instantiate Model

In [ ]:

from torchsummary import summary
input_shape = [(batch_size, seq_len, 2), (batch_size, Np)] # velocity-input UNION initial-state
summary(model, input_shape)

# Train model

In [ ]:
import tqdm

In [ ]:
a = tqdm.tqdm(range(10))
for i in a:
    i

In [ ]:
from tqdm import trange
from time import sleep
t = trange(100, desc='Bar desc', leave=True)
for i in t:
    t.set_description("Bar desc (file %i)" % i)
    t.refresh() # to show immediately the update
    sleep(0.01)
    
    
for i in tqdm

In [ ]:
from time import sleep

In [ ]:
for i in (t:=tqdm.trange(1000)):
    t.set_description("Bar desc (file %i)" % i, refresh=True)
    sleep(0.01)

In [ ]:
# train model
epochs = 10
steps_per_epoch = 200
model.fit(x=dg, epochs=epochs, steps_per_epoch=steps_per_epoch)

In [ ]:
model.RNN.weights